In [82]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
#염기서열은 g,c,a,t가 있다...
"""
모델 계획..
1. 0또는 1을 구하는 것이므로 classification을 해야한다.
2. classification은 결정트리, logistic알고리즘 등이 있다.
3. 과적합을 방지하고 정확도를 높이기 위해 앙상블 기법을 사용한다.
4. 모델을 test_data에 돌리고 정확도가 60% 이상이라면 kaggle에 올린다.


문제점.... 피처엔지니어링의 중요성.....
Q1. PR Seq, RT Seq의 DNA순서들을 어떻게 처리해야 텐서가 이해할 수 있을까...
A1. one-hot encoding이 좋은 방안이 될수 있을것 같다...

Q1-1. one-hot encode를 해도 데이터 길이가 일정하지 않다면 input_dim을 정해야 할때 문제가 생긴다.
A1-1_1. 데이터의 패턴을 찾는 알고리즘을 짜고 길이를 일정하게 맞춘다면?
A1-1_2. 가장 짧은 데이터를 기준으로 나머지를 잘라버린다..
A1-1_3. 그냥 무작위를 부족한 만큼 넣는다.

Q2. VL-t0, CD4-t0를 그냥 classification하면 둘다 숫자다 보니 정확도에 문제가 생긴다. 그럼 어떻게 처리해야할까?...
Q3. PR Seq가 없는 데이터가 있는데 이 데이터들은 또 어떻게 처리해야할까...
"""
# Predicting animal type based on various features
xy = pd.read_csv('./training_data.csv',dtype=None)

xy.drop(['PatientID'], axis=1,inplace=True)
#print(xy)

y_data = xy["Resp"]
x_PR_data = xy["PR Seq"]
x_RT_data = xy["RT Seq"]
x_data =xy.drop(["Resp","PR Seq","RT Seq"], axis=1)

y_data.to_numpy()  #병에 걸린지 아닌지
x_PR_data.to_numpy()
x_RT_data.to_numpy()
x_data.to_numpy()  # 사람의 염기서열이나 어러 지표들

#print(x_data)

#y_data.count()
#logistic classification
#pr과 나머지로 나눠준다...
#먼저 x_PR_data부터 train한다.

"""
tf_PR.model = tf.keras.Sequential()

tf_PR.model.add(tf.keras.layers.Dense(units=1, input_dim=1))  
tf_PR.model.add(tf.keras.layers.Activation('sigmoid'))


tf.model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.01), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_data, y_data, epochs=5000)

# Accuracy report
print("Accuracy: ", history.history['accuracy'][-1])

y_predict = tf.model.predict_classes(np.array([[5.1, 3.5 ,1.4, 0.2]]))
print(y_predict)
"""
enc = OneHotEncoder()

data=[[0,3,2,1,2,1,3,3,2,3]]
data = np.array(data)
data=data.reshape(-1,1)     #1차원 array를 2차원으로 만들어 준다.

enc.fit(data)   #enc에 one-hot encoder class를 가져오고, 
data_onehot = enc.transform(data).toarray()  #원핫인코딩을 진행한다.
#print(data_onehot)


#x_PR_data
def trans(x_PR_data):
    xprdata=[]
    for line in x_PR_data:
        now=[]
        #print("line=",line)
        try:
            for gen in line:
                if gen=="C":
                    now.append(0)
                elif gen=="T":
                    now.append(1)
                elif gen=="A":
                    now.append(2)
                else:
                    now.append(3)
            now=np.array(now)
            now=now.reshape(-1,1)
            enc.fit(now)
            now = enc.transform(now).toarray()
            xprdata.append(now)
            #print(now)
        except:
            print("burn")
    return np.array(xprdata)
x_PR_data=trans(x_PR_data)   #각A,G,T,C는 2차원 One-hot encode이고, 전체는 2차원의 합이므로 3차원이다.

#print(x_PR_data)
x_RT_data = trans(x_RT_data)
print("\n\n\n x_data==$$$$$$$$$$$$$")
x_RT_pd = pd.DataFrame(x_RT_data,columns="gtca")
print(x_RT_pd)


AttributeError: 'Series' object has no attribute 'vlaue_counts'